## Title tbd

The Mexican blind cave fish (Astyanax mexicanus) is a freshwater fish living in large sections of the southwest, including New Mexico, Texas and eastern and central Mexico [1]. Populations of this fish dwell in dark caves, resulting in de-pigmentation and degenerated or total loss of eyesight. Due to the commonality of the Astyanax and the two separate populations, surface and cave, it has been an ideal model for studying eye development (and lack thereof) [2]. Even though the cave population of the fish are blind, they are still able to avoid objects and hunt for prey by using their lateral line. This sensory system allows a fish to respond to changes in its surroundings by detecting flow stimuli, governed by hydrodynamics [3]. 


This project will explore the changes in flow around a blind cave fish as it glides towards a wall by using potential flow theory. Building off work done by Hassan (1991), this project will use a panel method on a NACA 0013 airfoil to represent the body of the fish. After establishing what the flow looks like surrounding the fish in open water, a wall will be added to the flow and the body of the fish will be moved progressively closer to the wall, looking at changes in streamlines and pressure coefficient along the body.  


In [ ]:
#First, let's import our libraries 

import numpy
import scipy 
from matplotlib import pyplot 
%matplotlib inline 

In [ ]:
#We will import the NACA 0013 airfoil from UIUC airfoil database 

In [ ]:
#Plot

In [ ]:
#We will use our vortex-source panel method to discretize the airfoil into panels

In [ ]:
#Plot

In [ ]:
#What does the fish look like in freestream

In [ ]:
#We will add a wall to our flow perpendicular to the fish

In [ ]:
#Plot

In [ ]:
#We will detect changes in the pressure coefficient as we move the fish closer to the wall

## Now let's look at what happens to a fish swimming parallel to the wall... 

In [ ]:
#We will move the location of our wall from perpendiciular to parallel 

In [ ]:
#Plot

In [ ]:
#We will detect changes in the pressure coefficient as we move the fish closer to the wall

## References

[1]
